In [257]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [258]:
df = pd.read_csv('/home/s2pac/ElbrusBootcamp/project/data/data.csv')
pd.set_option('display.max_columns', None)
df.head(3)

,Unnamed: 0,ID объявления,Количество комнат,Тип,Метро,Адрес,"Площадь, м2",Дом,Парковка,Цена,Телефоны,Описание,Ремонт,"Площадь комнат, м2",Балкон,Окна,Санузел,Можно с детьми/животными,Дополнительно,Название ЖК,Серия дома,"Высота потолков, м",Лифт,Мусоропровод,Ссылка на объявление
0,0,271271157,4,Квартира,м. Смоленская (9 мин пешком),"Москва, улица Новый Арбат, 27",200.0/20.0,"5/16, Монолитный",подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",+79166369231,Без комиссии для нанимателя! Бонус коллегам 12...,Дизайнерский,NaN,NaN,NaN,NaN,"Можно с детьми, Можно с животными","Мебель в комнатах, Мебель на кухне, Ванна, Душ...","Новый Арбат, 2010",NaN,3.0,"Пасс (4), Груз (1)",Да,https://www.cian.ru/rent/flat/271271157
1,1,271634126,4,Квартира,м. Смоленская (8 мин пешком),"Москва, улица Новый Арбат, 27",198.0/95.0/18.0,"5/16, Монолитно-кирпичный",подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",+79850608590,Лот 93107. Елена Анисимова.\n\nБонус агенту 50...,Дизайнерский,25 25 20 25,NaN,На улицу и двор,"Совмещенный (2), Раздельный (1)",Можно с детьми,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",Новый Арбат,NaN,3.5,"Пасс (1), Груз (1)",Нет,https://www.cian.ru/rent/flat/271634126
2,2,271173086,"4, Оба варианта",Квартира,м. Смоленская (7 мин пешком),"Москва, улица Новый Арбат, 27",200.0/116.0/4.0,5/16,подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...","+79672086536, +79099269384","ID 36380: Шикарная 4-х км. квартира в ЖК ""Нов...",Евроремонт,NaN,NaN,На улицу и двор,Совмещенный (3),Можно с детьми,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",Новый Арбат,NaN,3.2,Пасс (1),NaN,https://www.cian.ru/rent/flat/271173086


In [259]:
# Удаляю строки, если в адресе нет Москвы
df = df[df['Адрес'].str.lower().str.contains('москва')]


In [260]:
#Удаляю лишние колонки, которые не нужны для анализа
df.drop(columns=["Unnamed: 0", "Парковка", "Телефоны", "Описание", "Ссылка на объявление", "Серия дома", "Название ЖК", "Тип", "Высота потолков, м", "Мусоропровод", "Площадь комнат, м2", "Балкон", "Можно с детьми/животными", "Дополнительно", "Окна"], inplace=True)
df.head(1)

,ID объявления,Количество комнат,Метро,Адрес,"Площадь, м2",Дом,Цена,Ремонт,Санузел,Лифт
0,271271157,4,м. Смоленская (9 мин пешком),"Москва, улица Новый Арбат, 27",200.0/20.0,"5/16, Монолитный","500000.0 руб./ За месяц, Залог - 500000 руб., ...",Дизайнерский,NaN,"Пасс (4), Груз (1)"


In [261]:
#Проверяю на дубликаты
df.duplicated().sum().tolist()

0

**Обрабатываю Цену**


In [262]:
# Преобразую цену в целое число
df['Цена'] = df['Цена'].str.split('.').str[0]
df['Цена'] = df['Цена'].astype('int')

**Обрабатываю колонку Ремонт**

In [263]:
df["Ремонт"].unique()

array(['Дизайнерский', 'Евроремонт', 'Косметический', nan, 'Без ремонта'],
      dtype=object)

In [264]:
#0 - Косметический
#1 - Без ремонта
#2 - Евроремонт
#3 - Дизайнерский

df['Ремонт'] = df['Ремонт'].replace({
    'Косметический': 0,
    'Без ремонта': 1,
    'Евроремонт': 2,
    'Дизайнерский': 3
})

# Нахожу наиболее частый тип ремонта
repair_mode = df['Ремонт'].mode()[0]

# Заполняю пропуски
df['Ремонт'].fillna(repair_mode, inplace=True)

# Преобразую в целочисленный тип
df['Ремонт'] = df['Ремонт'].astype(int)


/tmp/ipykernel_63142/1091225742.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Ремонт'] = df['Ремонт'].replace({
/tmp/ipykernel_63142/1091225742.py:17: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Ремонт'].fillna(repair_mode, inplace=True)


**Обрабатываю Количество комнат** 

In [265]:
# Удаляю из колонки все символы после запятой
df['Количество комнат'] = df['Количество комнат'].astype(str).str.split(',').str[0]

# Преобразую колонку в числовой формат
df['Количество комнат'] = pd.to_numeric(df['Количество комнат'], errors='coerce')

# Создаю словарь значения количества комнат по цене
house_bathroom_types = df.groupby('Цена')['Количество комнат'].agg(
    lambda x: x.mode()[0] if len(x.mode()) > 0 and not pd.isna(x.mode()[0]) else None
).to_dict()

# Заполнения пропуски на основе цены
def fill_rooms_by_price(row):
    if pd.isna(row['Количество комнат']):
        price = row['Цена']
        if price in house_bathroom_types and not pd.isna(house_bathroom_types[price]):
            return house_bathroom_types[price]
    return row['Количество комнат']

# Применяю функцию заполнения
df['Количество комнат'] = df.apply(fill_rooms_by_price, axis=1)

# Если остались пропуски, заполняю наиболее частым значением
room_mode = df['Количество комнат'].mode()[0]
df['Количество комнат'] = df['Количество комнат'].fillna(room_mode)

#Приведу в числовой тип к int
df['Количество комнат'] = df['Количество комнат'].astype(int)

**Обрабатываю Дом**

In [266]:
#Удаляю из колонки Дом все символы до запятой
df['Дом'] = df['Дом'].str.split(',').str[1]

In [267]:
# Выделяю имя станции метро из строки
def extract_metro_station(metro_str):
    if pd.isna(metro_str):
        return None
    try:
        # Извлекаю станцию метро
        match = re.search(r'м\.\s+([^(]+)', metro_str)
        if match:
            return match.group(1).strip()
        return None
    except Exception:
        return None

# Создаю новую колонку с названиями станций метро
df['Станция_метро'] = df['Метро'].apply(extract_metro_station)

# Создаю словарь наиболее частых типов домов по станции метро
station_house_types = df.groupby('Станция_метро')['Дом'].agg(lambda x: x.mode()[0] if len(x.mode()) > 0 else None).to_dict()

# Заполняю пропуски на основе станции метро
def fill_house_by_station(row):
    if pd.isna(row['Дом']):
        station = row['Станция_метро']
        if station in station_house_types and not pd.isna(station_house_types[station]):
            return station_house_types[station]
    return row['Дом']

# Применяю функцию заполнения
df['Дом'] = df.apply(fill_house_by_station, axis=1)

# Если остались пропуски, заполняю общим наиболее частым значением
df['Дом'] = df['Дом'].fillna(df['Дом'].mode()[0])


In [268]:
df['Дом'].unique()

array([' Монолитный', ' Монолитно-кирпичный', ' Кирпичный', ' Панельный',
       ' Сталинский', ' старый фонд', ' Блочный', ' Деревянный',
       ' Щитовой'], dtype=object)

In [269]:
#0 - Панельный/Блочный/Щитовой
#1 - Кирпичный/Деревянный
#2 - Монолитный/Сталинский
#3 - Монолитно-кирпичный
#4 - старый фонд

df['Дом'] = df['Дом'].replace({
    ' Панельный': 0,
    ' Блочный': 0,
    ' Щитовой': 0,
    ' Кирпичный': 1,
    ' Деревянный': 1,
    ' Монолитный': 2,
    ' Сталинский': 2,
    ' Монолитно-кирпичный': 3,
    ' старый фонд': 4
})

# Нахожу наиболее частый тип Дома
house = df['Дом'].mode()[0]

# Заполняю пропуски
df['Дом'].fillna(house, inplace=True)

# Преобразую в целочисленный тип
df['Дом'] = df['Дом'].astype(int)

/tmp/ipykernel_63142/1842918352.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Дом'] = df['Дом'].replace({
/tmp/ipykernel_63142/1842918352.py:23: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Дом'].fillna(house, inplace=True)


**Обрабатываю метро**

In [270]:
# Преобразую колонку Метро в значение времени в минутах (пешком)
def preprocess_metro(metro_str):
    if pd.isna(metro_str):
        return None
    try:
        number = re.search(r'\((\d+)', metro_str)
        if not number:
            return None
        number = int(number.group(1))
        if 'на машине' in metro_str:
            return number * 4.9
        else:
            return number
    except Exception:
        return None

# Создаю новую колонку с названиями станций метро
df['Станция_метро'] = df['Метро'].apply(extract_metro_station)

df['Метро_время'] = df['Метро'].apply(preprocess_metro)

# Преобразую к числовому типу
df['Метро_время'] = pd.to_numeric(df['Метро_время'], errors='coerce')

# Заполняю пропущенные значения медианой по станциям
# Создаю словарь медиан для каждой станции
station_medians = df.groupby('Станция_метро')['Метро_время'].median().to_dict()

# Функция для заполнения пропусков на основе станции
def fill_metro_by_station(row):
    if pd.isna(row['Метро_время']) and not pd.isna(row['Станция_метро']):
        station = row['Станция_метро']
        if station in station_medians and not pd.isna(station_medians[station]):
            return station_medians[station]
    return row['Метро_время']

# Применяю функцию заполнения
df['Метро_время'] = df.apply(fill_metro_by_station, axis=1)

# Если все еще остались пропуски, заполняю их общей медианой
overall_median = df['Метро_время'].median()
df['Метро_время'] = df['Метро_время'].fillna(overall_median)

# Преобразую в целочисленный тип
df['Метро_время'] = df['Метро_время'].round().astype('int')

# Заменяю исходную колонку Метро на обработанную Метро_время
df['Метро'] = df['Метро_время']

# Удаляю временные колонки
df = df.drop(['Метро_время', 'Станция_метро'], axis=1)


**Обрабатываю Площадь**

In [271]:
# Обрабатываю площадь
df['Площадь, м2'] = df['Площадь, м2'].apply(lambda x: x.split('/')[0] if '/' in x else x)

#Преобразую в float
df['Площадь, м2'] =  df['Площадь, м2'].astype('float')

#Преобразую в int
df['Площадь, м2'] = df['Площадь, м2'].astype('int')

**Обрабатываю Санузел**

In [272]:
#0 - Совмещенный (1) / Раздельный (1) / Совмещенный (1), Раздельный (1) / Совмещенный (2) /
# Раздельный (2) / Совмещенный (1), Раздельный (2) / Совмещенный (2), Раздельный (2) / Совмещенный (2), Раздельный (2)

#1 - Совмещенный (2), Раздельный (1) / Совмещенный (3) / Совмещенный (1), Раздельный (3) /
# Совмещенный (3), Раздельный (2)

#2 - Совмещенный (3), Раздельный (3) / Совмещенный (2), Раздельный (3) / Совмещенный (3), Раздельный (1)

#3 - Раздельный (3) / Совмещенный (4) / Совмещенный (4), Раздельный (1)

#4 - Раздельный (4) / Совмещенный (4), Раздельный (2)

#5 - Совмещенный (2), Раздельный (4)


df['Санузел'] = df['Санузел'].replace({
    'Совмещенный (1)': 0,
    'Раздельный (1)': 0,
    'Совмещенный (1), Раздельный (1)': 0,
    'Совмещенный (2)': 0,
    'Раздельный (2) / Совмещенный (1)': 0,
    'Раздельный (2)': 0,
    'Совмещенный (1), Раздельный (2)': 0,
    'Совмещенный (2), Раздельный (2)': 0,
    'Совмещенный (2), Раздельный (1)': 1,
    'Совмещенный (3)': 1,
    'Совмещенный (1), Раздельный (3)': 1,
    'Совмещенный (3), Раздельный (2)': 1,
    'Совмещенный (3), Раздельный (3)': 2,
    'Совмещенный (2), Раздельный (3)': 2,
    'Совмещенный (3), Раздельный (1)': 2,
    'Раздельный (3)': 3,
    'Совмещенный (4)': 3,
    'Совмещенный (4), Раздельный (1)': 3,
    'Раздельный (4)': 4,
    'Совмещенный (4), Раздельный (2)': 4,
    'Совмещенный (2), Раздельный (4)': 5,


})

/tmp/ipykernel_63142/2362877580.py:16: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Санузел'] = df['Санузел'].replace({


In [273]:
# Создаю словарь санузлов по типу дома
house_bathroom_types = df.groupby('Дом')['Санузел'].agg(lambda x: x.mode()[0] if len(x.mode()) > 0 and not pd.isna(x.mode()[0]) else None).to_dict()

#Заполняю пропуски на основе типа дома
def fill_bathroom_by_house(row):
    if pd.isna(row['Санузел']):
        house = row['Дом']
        if house in house_bathroom_types and not pd.isna(house_bathroom_types[house]):
            return house_bathroom_types[house]
    return row['Санузел']

df['Санузел'] = df.apply(fill_bathroom_by_house, axis=1)

#Привожу к int
df['Санузел'] = df['Санузел'].astype(int)

In [274]:
df.drop(columns=["Адрес", "Лифт"], inplace=True)

In [275]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19737 entries, 0 to 23367
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype
---  ------             --------------  -----
 0   ID  объявления     19737 non-null  int64
 1   Количество комнат  19737 non-null  int64
 2   Метро              19737 non-null  int64
 3   Площадь, м2        19737 non-null  int64
 4   Дом                19737 non-null  int64
 5   Цена               19737 non-null  int64
 6   Ремонт             19737 non-null  int64
 7   Санузел            19737 non-null  int64
dtypes: int64(8)
memory usage: 1.4 MB


**Измененяю колонки с русского на английский**

In [276]:
# Переименовываю колонки
df.rename(columns={
    'ID  объявления': 'id_listing',
    'Количество комнат': 'number_of_rooms',
    'Метро': 'metro_on_foot',
    'Площадь, м2': 'area_m2',
    'Дом': 'building_info',
    'Цена': 'price',
    'Ремонт': 'renovation',
    'Санузел': 'bathroom',
}, inplace=True)

# Сохраняю измененния в файл
df.to_csv('/home/s2pac/ElbrusBootcamp/project/data/edited_data.csv', index=False)


In [277]:
# Выведем информацию о колонках
print("Колонки в датасете:")
print(df.columns.tolist())

Колонки в датасете:
['id_listing', 'number_of_rooms', 'metro_on_foot', 'area_m2', 'building_info', 'price', 'renovation', 'bathroom']


In [278]:
# Выведем первые 3 строки
print("Первые 3 строки:")
print(df.head(3))

Первые 3 строки:
   id_listing  number_of_rooms  metro_on_foot  area_m2  building_info   price  \
0   271271157                4              9      200              2  500000   
1   271634126                4              8      198              3  500000   
2   271173086                4              7      200              1  500000   

   renovation  bathroom  
0           3         0  
1           3         1  
2           2         1  


In [279]:
# Проверим наличие пропущенных значений
print("Количество пропущенных значений в каждой колонке:")
print(df.isna().sum())

Количество пропущенных значений в каждой колонке:
id_listing         0
number_of_rooms    0
metro_on_foot      0
area_m2            0
building_info      0
price              0
renovation         0
bathroom           0
dtype: int64


In [280]:
# Проверим типы данных
print("Типы данных:")
print(df.dtypes)

Типы данных:
id_listing         int64
number_of_rooms    int64
metro_on_foot      int64
area_m2            int64
building_info      int64
price              int64
renovation         int64
bathroom           int64
dtype: object


In [281]:
# Базовая статистика по численным колонкам
print("Статистика по численным данным:")
print(df.describe())

Статистика по численным данным:
         id_listing  number_of_rooms  metro_on_foot       area_m2  \
count  1.973700e+04     19737.000000   19737.000000  19737.000000   
mean   2.671149e+08         2.015352      12.869585     63.516796   
std    1.980106e+07         0.999502      14.270145     47.953921   
min    1.072986e+08         1.000000       0.000000      7.000000   
25%    2.712212e+08         1.000000       6.000000     38.000000   
50%    2.739284e+08         2.000000      10.000000     49.000000   
75%    2.746973e+08         3.000000      15.000000     70.000000   
max    2.750064e+08         6.000000     294.000000    811.000000   

       building_info         price    renovation      bathroom  
count   19737.000000  1.973700e+04  19737.000000  19737.000000  
mean        0.826417  8.656385e+04      1.138116      0.046917  
std         0.949988  1.281483e+05      1.181354      0.313731  
min         0.000000  5.000000e+03      0.000000      0.000000  
25%         0.000000 

In [282]:
print(f"Количество строк: {len(df)}")
print(f"Проверка на дубликаты: {df.duplicated().sum()} дубликатов")

Количество строк: 19737
Проверка на дубликаты: 0 дубликатов
